<a href="https://colab.research.google.com/github/leorex819-creator/fake-news-detection-multilingual_videos/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

folders = [
    "notebooks",
    "data/raw/text", "data/raw/images", "data/raw/videos",
    "data/processed/text", "data/processed/images", "data/processed/videos",
    "src", "models/text", "models/image", "models/video",
    "results/metrics", "results/graphs", "results/predictions",
    "utils"
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("✅ Professional project structure created")


✅ Professional project structure created


In [ ]:
open("data/raw/images/.gitkeep", "w").close()
open("data/raw/videos/.gitkeep", "w").close()
open("data/raw/text/.gitkeep", "w").close()


In [4]:
import os
print(os.getcwd())
print(os.listdir())


/content
['.config', 'notebooks', 'models', 'src', 'results', 'utils', 'data', 'sample_data']


In [5]:
import os

base_path = "/content/fake-news-detection-multilingual"

folders = [
    "notebooks",
    "data/raw/text",
    "data/raw/images",
    "data/raw/videos",
    "data/processed/text",
    "data/processed/images",
    "data/processed/videos",
    "src",
    "models/text",
    "models/image",
    "models/video",
    "results/metrics",
    "results/graphs",
    "results/predictions",
    "utils"
]

for folder in folders:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)

print("✅ Project structure created successfully")


✅ Project structure created successfully


In [6]:
os.chdir("/content/fake-news-detection-multilingual")
print(os.getcwd())


/content/fake-news-detection-multilingual


In [7]:
print(os.listdir("data"))
print(os.listdir("data/raw"))
print(os.listdir("data/raw/images"))


['processed', 'raw']
['videos', 'text', 'images']
[]


In [8]:
open("data/raw/images/.gitkeep", "w").close()
open("data/raw/videos/.gitkeep", "w").close()
open("data/raw/text/.gitkeep", "w").close()


In [9]:
!pip install spacy
!pip install sentence-transformers
!pip install langdetect
!pip install deep-translator


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=c703c9aebbc326f971571b180a97e76a9d72637974f8da82c6efb02c14f3ed8c
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.1 MB/s eta 0:00:00


In [10]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 94.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
import spacy
from langdetect import detect
from deep_translator import GoogleTranslator
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [12]:
# Load models
nlp = spacy.load("en_core_web_sm")
embedder = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
def normalize_text(text):
    lang = detect(text)

    if lang != "en":
        translated = GoogleTranslator(source=lang, target="en").translate(text)
        return translated, lang
    else:
        return text, "en"


In [14]:
def extract_claims(text):
    doc = nlp(text)
    claims = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 5]
    return claims


In [15]:
# Minimal training data (we will expand later)
train_claims = [
    "WHO approved the vaccine after clinical trials",
    "Vaccines contain tracking microchips",
    "Government released official statistics",
    "Drinking bleach cures diseases"
]

labels = [1, 0, 1, 0]  # 1 = Supported, 0 = Suspicious

X_train = embedder.encode(train_claims)

model = LogisticRegression()
model.fit(X_train, labels)


LogisticRegression()

In [16]:
##KNOWLEDGE-AWARE VERIFICATION LAYER
verified_knowledge = [
    "WHO approved COVID-19 vaccines after clinical trials",
    "Vaccines are tested for safety and effectiveness",
    "Government data is released by official agencies"
]

verified_embeddings = embedder.encode(verified_knowledge)

def verify_claim(claim):
    claim_emb = embedder.encode([claim])
    similarity = cosine_similarity(claim_emb, verified_embeddings)
    score = np.max(similarity)

    if score > 0.6:
        return "Verified by trusted knowledge", round(float(score), 2)
    else:
        return "No reliable verification found", round(float(score), 2)


In [17]:
##FINAL ANALYSIS PIPELINE
def analyze_news(text):
    normalized_text, original_lang = normalize_text(text)
    claims = extract_claims(normalized_text)
    embeddings = embedder.encode(claims)

    predictions = model.predict(embeddings)
    probabilities = model.predict_proba(embeddings)

    results = []
    for i, claim in enumerate(claims):
        status = "Supported" if predictions[i] == 1 else "Suspicious"
        confidence = round(max(probabilities[i]), 2)
        knowledge_result, similarity = verify_claim(claim)

        results.append({
            "claim": claim,
            "status": status,
            "confidence": confidence,
            "knowledge_check": knowledge_result,
            "similarity_score": similarity
        })

    return results, original_lang


In [18]:
news = """
WHO approved the vaccine after multiple clinical trials.
The vaccine causes infertility.
"""

output, lang = analyze_news(news)

for o in output:
    print(o)


{'claim': 'WHO approved the vaccine after multiple clinical trials.', 'status': 'Supported', 'confidence': np.float64(0.55), 'knowledge_check': 'Verified by trusted knowledge', 'similarity_score': 0.76}
{'claim': 'The vaccine causes infertility.', 'status': 'Suspicious', 'confidence': np.float64(0.51), 'knowledge_check': 'No reliable verification found', 'similarity_score': 0.55}


In [19]:
news_tamil = "இந்த தடுப்பூசி மக்களை கட்டுப்படுத்த மைக்ரோசிப் கொண்டுள்ளது"

output, lang = analyze_news(news_tamil)

for o in output:
    print(o)


{'claim': 'This vaccine contains a microchip to control the population', 'status': 'Suspicious', 'confidence': np.float64(0.53), 'knowledge_check': 'No reliable verification found', 'similarity_score': 0.57}


In [21]:
!pip install moviepy
!pip install openai-whisper


In [22]:
from moviepy.editor import VideoFileClip
import whisper
import os


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [23]:
project video

SyntaxError: invalid syntax (ipython-input-3033180241.py, line 1)

In [24]:
news_video.mp4


NameError: name 'news_video' is not defined

In [25]:
news_video.mp4


NameError: name 'news_video' is not defined

In [26]:
def extract_audio(video_path, audio_path="audio.wav"):
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)
    return audio_path


In [27]:
audio_file = extract_audio("news_video.mp4")


OSError: MoviePy error: the file news_video.mp4 could not be found!
Please check that you entered the correct path.

In [28]:
from moviepy.editor import VideoFileClip

def extract_audio(video_path, audio_path="audio.wav"):
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)
    return audio_path

audio_file = extract_audio("/content/news_video.mp4.mp4")
print("✅ Audio extracted to:", audio_file)


MoviePy - Writing audio in audio.wav


MoviePy - Done.
✅ Audio extracted to: audio.wav


In [32]:
model_whisper = whisper.load_model("base")

def speech_to_text(audio_path):
    result = model_whisper.transcribe(audio_path)
    return result["text"]


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 101MiB/s]


In [33]:
spoken_text = speech_to_text(audio_file)
print(spoken_text)



  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



 This is BBC News, The Headlines. A mass polio vaccination campaign across most of Northern Gaza is halted due to the bomb ablut. Israel bombs the centre of the historic Lebanese city of Tyre, civil defence workers say the Israelis are turning it into a kill zone. And there are fears over the lungs of planet Earth as the Amazon suffers its worst forest fires in two decades.


In [34]:
results, detected_lang = analyze_news(spoken_text)

for r in results:
    print("\nClaim:", r["claim"])
    print("Status:", r["status"])
    print("Confidence:", r["confidence"])
    print("Knowledge Check:", r["knowledge_check"])



Claim: This is BBC News, The Headlines.
Status: Supported
Confidence: 0.51
Knowledge Check: No reliable verification found

Claim: A mass polio vaccination campaign across most of Northern Gaza is halted due to the bomb ablut.
Status: Suspicious
Confidence: 0.52
Knowledge Check: No reliable verification found

Claim: Israel bombs the centre of the historic Lebanese city of Tyre, civil defence workers say the Israelis are turning it into a kill zone.
Status: Suspicious
Confidence: 0.54
Knowledge Check: No reliable verification found

Claim: And there are fears over the lungs of planet Earth as the Amazon suffers its worst forest fires in two decades.
Status: Suspicious
Confidence: 0.52
Knowledge Check: No reliable verification found


In [35]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.4 MB/s eta 0:00:00


In [37]:
import streamlit as st
import tempfile
from moviepy.editor import VideoFileClip
import whisper

# Reuse models from earlier
model_whisper = whisper.load_model("base")

st.set_page_config(page_title="Mindful Fake News Detector", layout="centered")

st.title("🧠 Mindful Fake News Detection System")
st.write("Explainable • Multilingual • Text & Video")

option = st.radio("Choose input type:", ["Text", "Video"])

# -------- TEXT INPUT --------
if option == "Text":
    text_input = st.text_area("Paste news content here")

    if st.button("Analyze Text") and text_input:
        results, lang = analyze_news(text_input)

        st.subheader(f"Detected Language: {lang}")
        for r in results:
            st.markdown(f"### 🧾 Claim: {r['claim']}")
            st.write("**Status:**", r["status"])
            st.write("**Confidence:**", r["confidence"])
            st.write("**Knowledge Check:**", r["knowledge_check"])
            st.write("---")

# -------- VIDEO INPUT --------
if option == "Video":
    video_file = st.file_uploader("Upload a news video", type=["mp4"])

    if video_file and st.button("Analyze Video"):
        with tempfile.NamedTemporaryFile(delete=False) as temp_video:
            temp_video.write(video_file.read())
            video_path = temp_video.name

        video = VideoFileClip(video_path)
        audio_path = video_path + ".wav"
        video.audio.write_audiofile(audio_path)

        result = model_whisper.transcribe(audio_path)
        spoken_text = result["text"]

        st.subheader("Extracted Speech")
        st.write(spoken_text)

        results, lang = analyze_news(spoken_text)

        st.subheader(f"Detected Language: {lang}")
        for r in results:
            st.markdown(f"### 🧾 Claim: {r['claim']}")
            st.write("**Status:**", r["status"])
            st.write("**Confidence:**", r["confidence"])
            st.write("**Knowledge Check:**", r["knowledge_check"])
            st.write("---")


2026-02-05 08:01:52.781 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-05 08:01:52.782 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-05 08:01:52.959 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2026-02-05 08:01:52.960 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-05 08:01:52.962 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-05 08:01:52.967 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-05 08:01:52.968 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [38]:
import joblib
joblib.dump(model, "fake_news_model.pkl")


['fake_news_model.pkl']